In [79]:
import numpy as np
import pandas as pd
import os
import jinja2

In [80]:
path = '../datasets/FBref/WC_2019'

In [81]:
keeper = []

In [82]:
for root, dirs, files in os.walk(path):
    for file in files:
        if 'keeper' in file:
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path, skiprows = 1)
            df['Team'] = file.split()[0]
            keeper.append(df)

In [83]:
combined_keep = pd.concat(keeper, ignore_index = True)

In [84]:
combined_keep.head(5)

,Player,Age,Min,SoTA,GA,Saves,Save%,PSxG,Cmp,Att,...,AvgLen,Att.2,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,AvgDist,Team
0,Sari van Veenendaal,29-095,90,10,2,8,90.0,2.7,2,16,...,42.7,8,75.0,41.4,16,1,6.3,3,18.5,Netherlands
1,Alyssa Naeher,31-078,90,1,0,1,100.0,0.0,4,13,...,38.9,5,100.0,54.2,6,0,0.0,2,19.3,USA
2,Karen Bardsley,34-256,90,3,0,3,100.0,1.0,4,9,...,33.5,7,71.4,40.4,6,0,0.0,0,8.3,England
3,Ingrid Hjelmseth,39-078,90,6,3,2,50.0,3.0,1,3,...,23.8,11,18.2,27.5,14,0,0.0,2,16.6,Norway
4,Sarah Bouhaddi,32-254,90,8,2,6,75.0,1.7,5,12,...,37.4,4,75.0,42.5,5,0,0.0,5,27.6,France


In [85]:
grouped_keeper = combined_keep[['Player', 'SoTA', 'GA']].groupby('Player').sum()

In [86]:
keeper_game_count = combined_keep['Player'].value_counts()

In [87]:
grouped_keeper['Games'] = keeper_game_count

In [88]:
grouped_keeper.reset_index(inplace = True)

In [89]:
grouped_keeper['NSPG'] = grouped_keeper.apply(lambda row: (row['SoTA']-row['GA'])/row['Games'], axis = 1)

In [90]:
grouped_keeper = pd.merge(grouped_keeper, combined_keep[['Player', 'Team']].drop_duplicates(), on='Player',how = 'left')

In [91]:
grouped_keeper.sort_values(by = 'NSPG', ascending = False)

,Player,SoTA,GA,Games,NSPG,Team
11,Lydia Williams,8,1,1,7.00,Australia
10,Laura Giuliani,15,2,2,6.50,Italy
14,Sarah Bouhaddi,15,3,2,6.00,France
15,Sari van Veenendaal,21,3,4,4.50,Netherlands
5,Carly Telford,11,4,2,3.50,England
0,Almuth Schult,8,2,2,3.00,Germany
12,Peng Shimeng,5,2,1,3.00,China
8,Ingrid Hjelmseth,10,4,2,3.00,Norway
3,Ayaka Yamashita,5,2,1,3.00,Japan
7,Hedvig Lindahl,14,3,4,2.75,Sweden


In [92]:
summary = []

In [93]:
for root, dirs, files in os.walk(path):
    for file in files:
        if 'summary' in file:
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path, skiprows = 1)
            df = df.iloc[:-1] 
            df['Team'] = file.split('-')[0]
            summary.append(df)

In [94]:
combined_summary = pd.concat(summary, ignore_index = True)

In [95]:
combined_summary.head(20)

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,GCA,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att.1,Succ,Team
0,Lineth Beerensteyn,21.0,FW,22-269,90,0,0,0,0,1,...,0.0,8.0,12.0,66.7,0.0,13.0,1.0,2.0,1.0,Netherlands
1,Lieke Martens,11.0,LW,26-203,69,0,0,0,0,0,...,0.0,12.0,24.0,50.0,3.0,14.0,1.0,3.0,2.0,Netherlands
2,Jill Roord,19.0,"AM,LW",22-076,21,0,0,0,0,2,...,0.0,9.0,11.0,81.8,2.0,10.0,0.0,1.0,1.0,Netherlands
3,Daniëlle van de Donk,10.0,"RW,WB",27-336,90,0,0,0,0,0,...,0.0,19.0,35.0,54.3,0.0,30.0,1.0,2.0,1.0,Netherlands
4,Vivianne Miedema,9.0,"AM,FW",22-357,90,0,0,0,0,0,...,0.0,16.0,24.0,66.7,3.0,12.0,3.0,5.0,2.0,Netherlands
5,Sherida Spitse,8.0,"DM,CM",29-039,90,0,0,0,0,2,...,0.0,32.0,47.0,68.1,3.0,25.0,0.0,1.0,1.0,Netherlands
6,Jackie Groenen,14.0,"DM,CM",24-202,90,0,0,0,0,0,...,0.0,28.0,38.0,73.7,8.0,28.0,1.0,5.0,1.0,Netherlands
7,Dominique Janssen,20.0,"LB,CB",24-171,90,0,0,0,0,0,...,0.0,35.0,52.0,67.3,4.0,24.0,3.0,2.0,1.0,Netherlands
8,Stefanie van der Gragt,3.0,CB,26-325,90,0,0,0,0,0,...,0.0,26.0,35.0,74.3,2.0,19.0,0.0,0.0,0.0,Netherlands
9,Anouk Dekker,6.0,CB,32-234,72,0,0,0,0,0,...,0.0,23.0,26.0,88.5,1.0,15.0,0.0,0.0,0.0,Netherlands


In [96]:
game_count = combined_summary['Player'].value_counts()

In [98]:
grouped_goals = combined_summary[['Player', 'Gls']].groupby('Player').sum()

grouped_goals['Games'] = game_count

grouped_goals.reset_index(inplace = True)

grouped_goals['Goals/game']= (grouped_goals['Gls']/grouped_goals['Games']).round(2)

grouped_goals = pd.merge(grouped_goals, combined_summary[['Player', 'Team']].drop_duplicates(), on='Player',how = 'left')

grouped_goals = grouped_goals.rename(columns= {'Gls':'Goals'})

grouped_goals.sort_values(by = 'Goals/game', ascending = False).head(10).style.background_gradient(subset = ["Goals/game"], 
                             cmap = "Blues")

,Player,Goals,Games,Goals/game,Team
168,Megan Rapinoe,5,3,1.670000,USA
68,Elise Kellond-Knight,1,1,1.000000,Australia
109,Jennifer Hermoso,1,1,1.000000,Spain
242,Yui Hasegawa,1,1,1.000000,Japan
221,Thaisa,1,1,1.000000,Brazil
69,Ellen White,3,4,0.750000,England
132,Lea Schüller,1,2,0.500000,Germany
142,Lina Magull,1,2,0.500000,Germany
202,Sara Däbritz,1,2,0.500000,Germany
209,Sofia Jakobsson,2,4,0.500000,Sweden


In [68]:
grouped_goals.sort_values(by = 'Gls', ascending = False).head(10)

,Player,Gls,Games,Goals/game,Team
168,Megan Rapinoe,5,3,1.67,USA
69,Ellen White,3,4,0.75,England
216,Stina Blackstenius,2,4,0.50,Sweden
209,Sofia Jakobsson,2,4,0.50,Sweden
139,Lieke Martens,2,4,0.50,Netherlands
104,Jackie Groenen,1,4,0.25,Netherlands
112,Jill Scott,1,4,0.25,England
30,Aurora Galli,1,2,0.50,Italy
150,Lucy Bronze,1,4,0.25,England
221,Thaisa,1,1,1.00,Brazil


In [69]:
grouped_touches = combined_summary[['Player', 'Touches']].groupby('Player').sum()

grouped_touches['Games'] = game_count

grouped_touches.reset_index(inplace = True)

grouped_touches['Touches/game']= (grouped_touches['Touches']/grouped_touches['Games']).round(2)

grouped_touches = pd.merge(grouped_touches, combined_summary[['Player', 'Team']].drop_duplicates(), on='Player',how = 'left')

grouped_touches.sort_values(by = 'Touches/game', ascending = False).head(10).head(10).style.background_gradient(subset = ["Touches/game"], 
                             cmap = "Blues")

,Player,Touches,Games,Touches/game,Team
73,Emily van Egmond,111.000000,1,111.000000,Australia
220,Tamires,104.000000,1,104.000000,Brazil
70,Ellie Carpenter,103.000000,1,103.000000,Australia
246,Zhang Rui,98.000000,1,98.000000,China
150,Lucy Bronze,387.000000,4,96.750000,England
62,Desiree van Lunteren,378.000000,4,94.500000,Netherlands
5,Alex Greenwood,178.000000,2,89.000000,England
211,Sophie Schmidt,87.000000,1,87.000000,Canada
213,Steph Catley,85.000000,1,85.000000,Australia
4,Alanna Kennedy,85.000000,1,85.000000,Australia


In [99]:
grouped_pass = combined_summary[['Player', 'Cmp']].groupby('Player').sum()

grouped_pass['Games'] = game_count

grouped_pass.reset_index(inplace = True)

grouped_pass['Passes/game']= (grouped_pass['Cmp']/grouped_pass['Games']).round(2)

grouped_pass = pd.merge(grouped_pass, combined_summary[['Player', 'Team']].drop_duplicates(), on='Player',how = 'left')

grouped_pass = grouped_pass.rename(columns= {'Cmp':'Completed Passes'})

grouped_pass.sort_values(by = 'Passes/game', ascending = False).head(10).head(10).style.background_gradient(subset = ["Passes/game"], 
                             cmap = "Blues")

,Player,Completed Passes,Games,Passes/game,Team
73,Emily van Egmond,73.000000,1,73.000000,Australia
4,Alanna Kennedy,72.000000,1,72.000000,Australia
246,Zhang Rui,67.000000,1,67.000000,China
207,Shelina Zadorsky,65.000000,1,65.000000,Canada
213,Steph Catley,65.000000,1,65.000000,Australia
70,Ellie Carpenter,62.000000,1,62.000000,Australia
160,Maren Mjelde,124.000000,2,62.000000,Norway
150,Lucy Bronze,247.000000,4,61.750000,England
0,Abbie McManus,61.000000,1,61.000000,England
162,Marina Hegering,119.000000,2,59.500000,Germany


In [71]:
defence = []

In [72]:
for root, dirs, files in os.walk(path):
    for file in files:
        if 'defensive' in file:
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path, skiprows = 1)
            df = df.iloc[:-1] 
            df['Team'] = file.split('-')[0]
            defence.append(df)

In [73]:
combined_defence = pd.concat(defence, ignore_index = True)

In [100]:
grouped_defence = combined_defence[['Player', 'TklW']].groupby('Player').sum()

grouped_defence['Games'] = game_count

grouped_defence.reset_index(inplace = True)

grouped_defence['Tackle/game']= (grouped_defence['TklW']/grouped_defence['Games']).round(2)

grouped_defence = pd.merge(grouped_defence, combined_defence[['Player', 'Team']].drop_duplicates(), on='Player',how = 'left')

grouped_defence = grouped_defence.rename(columns= {'TklW':'Tackle Won'})

grouped_defence.sort_values(by = 'Tackle/game', ascending = False).head(10).head(10).style.background_gradient(subset = ["Tackle/game"], 
                             cmap = "Blues")

,Player,Tackle Won,Games,Tackle/game,Team
137,Letícia Santos,5,1,5.000000,Brazil
70,Ellie Carpenter,5,1,5.000000,Australia
166,Marta Corredera,5,1,5.000000,Spain
220,Tamires,4,1,4.000000,Brazil
75,Eugénie Le Sommer,8,2,4.000000,France
231,Virginia Torrecilla,4,1,4.000000,Spain
94,Hanna Glas,13,4,3.250000,Sweden
54,Crystal Dunn,12,4,3.000000,USA
211,Sophie Schmidt,3,1,3.000000,Canada
31,Aya Sameshima,3,1,3.000000,Japan


In [274]:
#save for tableau
grouped_keeper.to_csv('../Datasets/cleaned_data/2019/grouped_keeper.csv', index = False)

In [275]:
grouped_goals.to_csv('../Datasets/cleaned_data/2019/grouped_goals.csv', index = False)

In [276]:
grouped_touches.to_csv('../Datasets/cleaned_data/2019/grouped_tounches.csv', index = False)

In [277]:
grouped_pass.to_csv('../Datasets/cleaned_data/2019/grouped_pass.csv', index = False)

In [278]:
grouped_defence.to_csv('../Datasets/cleaned_data/2019/grouped_defence.csv', index = False)